In [ ]:
from __future__ import print_function
import cmlapi
from cmlapi.rest import ApiException
from pprint import pprint
import json, secrets, os, time
from datetime import datetime
import random
import mlflow

cml_client = cmlapi.default_client()
client=cmlapi.default_client()
experiment_name = "Churn Model Tuning"
username = os.environ["PROJECT_OWNER"]
model_name =  os.getenv("REGISTERED_MODEL_NAME") or "Customer Churn Model Endpoint - MLOps API"

In [ ]:

try : 
    #List all the experiments in a given project of a certain name
    search_filter= { "name" : experiment_name}
    project_id = os.environ["CDSW_PROJECT_ID"]
    search = json.dumps(search_filter)
    api_response = cml_client.list_experiments( project_id = project_id , search_filter = search)
    assert len(api_response.experiments) == 1
    experiment = api_response.experiments[0]
    pprint(experiment)
    
    threshold = 0.71
    # we use now MLFlow api to filter the runs for metrics ,since they are not implemented in V2 yet
    exp_run_list =mlflow.search_runs(
        experiment.id,  
#        filter_string="metric.test_score < 0.71 and metric.test_score > 0.6", # filters can also take complex logic like this
        filter_string=f"metric.test_score > {threshold}",
        order_by=["metrics.training_score DESC", "start_time DESC"]
    )

    # lets take the first run with the highest training score above threshold
    best_run=exp_run_list.iloc[0]
    print(best_run)
    
except ApiException as e :
    print("Exception when calling CMLServiceapi -> list experiments: $s\n" %e)

In [ ]:

def get_best_experiment_run(experiment_name, threshold =0.5, **kwargs): 
    try : 
        #List all the experiments in a given project of a certain name
        search_filter = {"name": experiment_name}
        search = json.dumps(search_filter)        
        # Let us see if we have Project Id 
        project_id=""
        filter_string=""
        
        # We will take our current project as default project 
        if len(kwargs) == 0 :
            project_id = os.environ["CDSW_PROJECT_ID"]
        
        # Project Id and filter string will be overwritten if provided
        for key, value in kwargs.items() : 
            if  key == "project_id" :
                project_id = kwargs[key]
            elif key == "filter_string" : 
                filter_string = kwargs[key]
                search = json.dumps(filter_string)
        api_response = cml_client.list_experiments( project_id = project_id , search_filter = search)
        assert len(api_response.experiments) == 1
        experiment = api_response.experiments[0]
        pprint(experiment)
    except ApiException as e :
        print("Exception when calling CMLServiceapi -> list experiments: $s\n" %e)

    try : 
        # Check if filter_string has been provided if not create our own with the threshold
        # filter_string="metric.test_score < 0.71 and metric.test_score > 0.6", # filters can also take multiple criteria like this when passing
        if(len(filter_string) == 0):
            filter_string = f"metric.test_score > {threshold}"
            
        # we use now MLFlow api to filter the runs for metrics ,since they are not implemented in V2 yet
        exp_run_list =mlflow.search_runs(
            experiment.id,  

            filter_string=filter_string,
            order_by=["metrics.training_score DESC", "start_time DESC"]
        )

        # lets take the first run with the highest training score above threshold
        best_run=exp_run_list.iloc[0]
        print(best_run)
        return(best_run)
    except Exception as err:
        print(f"Error in Mlflow Experiment search_runs {err=}, {type(err)=}")
        raise


In [ ]:
best_run = get_best_experiment_run(experiment_name="Churn Model Tuning", threshold=0.7)

In [ ]:

# for testing only
# project_id = os.environ["CDSW_PROJECT_ID"]
# session_id = secrets.token_hex(nbytes=4)

In [ ]:
# model_name = 'churn_model-' + username + "-" + session_id # For testing purposes ( use a session id to generate a unique model)
model_path = '{0}/model'.format(best_run.artifact_uri)
print(model_path)
CreateRegisteredModelRequest = {
                                "project_id": project_id, 
                                "experiment_id" : best_run.experiment_id,
                                "run_id": best_run.run_id, 
                                "model_name": model_name, 
                                "model_path": model_path, 
                                "visibility": "PUBLIC"   }
                               
print(CreateRegisteredModelRequest)
try : 
    #Register the best performing model either as a new entry in the Model Registry or as a new verion
    api_response = cml_client.create_registered_model(CreateRegisteredModelRequest)
    # This prints the metadata after we register the model including the model version details
    pprint(api_response)
except ApiException as e:
    print("Exception when calling CMLServiceApi->create_registered_model: %s\n" % e)
    
    

In [ ]:
print(api_response.model_id,"---", api_response.model_versions[0].model_version_id, api_response.name)

In [ ]:
# get a list of Registered Models with the name we have chosen! 
# starting from scratch here, because the deployment could be done by someone else who has not registered the model and all they have is the model
#name is picked from the print statement above and assumption is we have had a successful model registration

from __future__ import print_function
import cmlapi
from cmlapi.rest import ApiException
from pprint import pprint
import json, secrets, os, time
from datetime import datetime
import random
import mlflow

cml_client = cmlapi.default_client()


try : 
    #List all the experiments in a given project of a certain name
    search_filter= { "model_name" : model_name}
    
    #Uncomment this to test working against multiple versions
    project_id = os.environ["CDSW_PROJECT_ID"]
    search = json.dumps(search_filter)
    registered_model_list = cml_client.list_registered_models( search_filter = search)
    pprint(registered_model_list)
    # assuming only one model is returned

    model_id = registered_model_list.models[0].model_id
    print(model_id)
    # sort by -version_number gets us a list of models with version number descending first.
    registered_model = cml_client.get_registered_model(model_id, sort="-version_number")
    pprint(registered_model)

    # lets us create 3 model variables #deploy_model_Id and #deploy_version_id
    deploy_model_id =registered_model.model_id
    deploy_version_id = registered_model.model_versions[0].model_version_id
    print(deploy_model_id, deploy_version_id)
    
except ApiException as e:
    print("Exception when calling CMLServiceApi->list_registered_models: %s\n" % e)    


In [ ]:

def create_model_for_deployment(client, projectId, modelName, modelId, description = "Churn Model via API"):
    """
    Method to create a model  for deployment from Model Registry, if a model exists with the same name it wirl 
    """

    # first check if the Model with that name exists
    search_filter = {"name": modelName}
    search = json.dumps(search_filter)    

    api_response=client.list_all_models(search_filter=search)

    if len(api_response.models) != 0 : 
        print("Model with this name already deployed")
        pprint(api_response.models[0])
        modelObj = api_response.models[0]
    
    else :
        project_id = os.environ["CDSW_PROJECT_ID"]
        CreateModelRequest = {
                                "project_id": projectId, 
                                "name" : modelName,
                                "description": description, 
                                "disable_authentication": True,
                                "registered_model_id": modelId
                             }

        try:
            # Create a model.
            api_response = client.create_model(CreateModelRequest, projectId)
            pprint(api_response)
            modelObj = api_response
        except ApiException as e:
            print("Exception when calling CMLServiceApi->create_model: %s\n" % e)
            raise
        
    return modelObj

In [ ]:

def create_modelBuild_for_deployment(client, projectId, modelVersionId, modelCreationId):
    """
    Method to create a Model build
    """
    
    print(f'model version id:{modelVersionId} and model Creation Id :{modelCreationId}')
    
    # Create Model Build
    CreateModelBuildRequest = {
                                "registered_model_version_id": modelVersionId, 
                                "runtime_identifier": "docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-workbench-python3.9-standard:2023.08.2-b8",
                                "comment": "invoking model build",
                                "model_id": modelCreationId
                              }

    try:
        # Create a model build.
        api_response = client.create_model_build(CreateModelBuildRequest, projectId, modelCreationId)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling CMLServiceApi->create_model_build: %s\n" % e)

    return api_response

In [ ]:
import time

import os
import json
import string
import cmlapi
from src.api import ApiUtility
import cdsw
import sys
from datetime import datetime

# lets us get a Handle to API 
client = cmlapi.default_client()
# project_id = os.environ["CDSW_PROJECT_ID"]
# projects = client.list_projects(search_filter=json.dumps({"name": "LLM-demo-on-CML"}))
# project = projects.projects[0] # assuming only one project is returned by the above query

project_id = os.environ["CDSW_PROJECT_ID"]

# create a model request
#model_body = cmlapi.CreateModelRequest(project_id=project_id, name="Churn Model API Endpoint - API", description="Deploy Churn Model with the API Endpoint")
model = create_model_for_deployment(client = client , projectId = project_id, description="Churn Model API Endpoint - API12",modelName = model_name,modelId=deploy_model_id )

In [ ]:
#take the model creation id from the model value above
model_build = create_modelBuild_for_deployment(client = client , projectId = project_id, modelVersionId=deploy_version_id, modelCreationId =model.id)
# model_build = create_modelBuild_for_deployment(client = client , projectId = project_id, modelVersionId=deploy_version_id, modelCreationId =model.registered_model_id)

In [ ]:
# create a model request
runtime_details='docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-workbench-python3.9-standard:2023.05.2-b7'

start_time = datetime.now()
print(start_time.strftime("%H:%M:%S"))


In [ ]:
# # Model is getting Built as a container image
# model_build = client.create_model_build(model_build_body, project.id, model.id)
while model_build.status not in ["built", "build failed"]:
    print("waiting for model to build...")
    time.sleep(10)
    model_build = cml_client.get_model_build(project_id ,model.id, model_build.id)
    if model_build.status == "build failed" :
        print("model build failed, see UI for more information")
        sys.exit(1)
        
build_time = datetime.now()   
print(f"Time required for building model (sec): {(build_time - start_time).seconds}")
print("model built successfully!")

In [ ]:
# Model is getting deployed as a container image
model_deployment_body = cmlapi.CreateModelDeploymentRequest(project_id=project_id, model_id=model.id, build_id=model_build.id, cpu=2, memory=4)
model_deployment = cml_client.create_model_deployment(model_deployment_body, project_id, model.id, model_build.id)

while model_deployment.status not in ["stopped", "failed", "deployed"]:
    print("waiting for model to deploy...")
    time.sleep(10)
    model_deployment = cml_client.get_model_deployment(project_id, model.id, model_build.id, model_deployment.id)

curr_time = datetime.now()

if model_deployment.status != "deployed":
    print("model deployment failed, see UI for more information")
    sys.exit(1)

if model_deployment.status == "deployed" :
    print(f"Time required for deploying model (sec): {(curr_time - start_time).seconds}")
print("model deployed successfully!")

## Inferencing from the Deployed Model

In [ ]:
import os
import json
import string
import cmlapi
from src.api import ApiUtility
import cdsw

# let us try inferencing from this model  for 5 customers with features bewlo
model_Input =  {"inputs": [[0.0, 0.0, 1.0, 1.0, 58.0, 1.0, 0.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.0, 1.0, 20.5, 1191.4], [0.0, 0.0, 1.0, 0.0, 50.0, 1.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 2.0, 1.0, 0.0, 0.0, 75.7, 3876.2], [0.0, 0.0, 1.0, 0.0, 55.0, 1.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 90.15, 4916.95], [0.0, 0.0, 0.0, 0.0, 16.0, 1.0, 2.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 1.0, 2.0, 88.45, 1422.1], [0.0, 0.0, 1.0, 0.0, 8.0, 1.0, 0.0, 1.0, 0.0, 2.0, 2.0, 0.0, 2.0, 2.0, 0.0, 0.0, 2.0, 101.15, 842.9]]}

model_name =  os.getenv("REGISTERED_MODEL_NAME") or "Customer Churn Model Endpoint - MLOps API"
client = cmlapi.default_client()
project_id = os.environ["CDSW_PROJECT_ID"]
#client.list_models(project_id)

# You can use an APIV2-based utility to access the latest model's metadata. For more, explore the src folder
apiUtil = ApiUtility()
Model_AccessKey = apiUtil.get_latest_deployment_details(model_name=model_name)["model_access_key"]
#Deployment_CRN = apiUtil.get_latest_deployment_details(model_name="Churn Model API Endpoint - API")["latest_deployment_crn"]
print(Model_AccessKey)

In [ ]:
#calling the model
response = cdsw.call_model(model_access_key=Model_AccessKey,ipt=model_Input)

pprint(response)

In [ ]:
%run 8A_register_model.py

In [ ]:
%run 8B_deploy_registered_model.py
